This assignment revolves some experiments which take few minutes to finish calculation and plotting. I'm sure that the code works well and will output the appropriate results as expected. Please do not interrupt while it is running.<br/>
Thank you!

In [29]:
import numpy as np
import igl
import meshplot as mp
import math
import time

# Setting up the Constraints

Define a function to retreive the index of the colosest point in _points_ to _p_ using brute-force for loop.

In [30]:
def find_closest_point(p, points):
    closest_i = 0
    min_val = 10000
    for i in range(points.shape[0]):
        d = np.linalg.norm(p - points[i])
        if d < min_val:
            min_val = d
            closest_i = i
    return closest_i

# Implement a spatial index to accelerate neighbor calculations

In [31]:
def set_newgrid(step, p):
    mmax = np.max(p, axis = 0)
    mmin = np.min(p, axis = 0)
    dim = 1.1 * (mmax - mmin)
    diag = np.linalg.norm(dim)
    grid_step = step * diag
    units_size = np.zeros(3)
    units_size[0] = math.ceil(dim[0] / grid_step)
    units_size[1] = math.ceil(dim[1] / grid_step)
    units_size[2] = math.ceil(dim[2] / grid_step)
    newgrid_size = int(np.prod(units_size))
    newgrid = [[] for i in range(newgrid_size)]
    
    for i in range(p.shape[0]):
        p_dist = np.zeros(3)
        p_dist[0] = math.floor(((p[i] - mmin) / grid_step)[0])
        p_dist[1] = math.floor(((p[i] - mmin) / grid_step)[1])
        p_dist[2] = math.floor(((p[i] - mmin) / grid_step)[2])
        index = int(p_dist[0] + units_size[0] * p_dist[1] + units_size[0] * units_size[1] * p_dist[2])
        newgrid[index].append(i)
    
    return newgrid, units_size, grid_step

To improve the efficiency by implementing a simple spatial index, we bin vertices into their enclosing grid cells and restrict the neighbor queries to visit only the grid cells that could possibly satisfy the query.<br/>
Define a new method to accelerate neighbor calculations.

In [32]:
def find_closed_point(p, points, newgrid, units_size, grid_step):
    closest_i = 0
    min_val = 10000
    mmin = np.min(points, axis = 0)
    p_dist = np.zeros(3)
    p_dist[0] = math.floor(((p - mmin) / grid_step)[0])
    p_dist[1] = math.floor(((p - mmin) / grid_step)[1])
    p_dist[2] = math.floor(((p - mmin) / grid_step)[2])

    check_distance = 1
    min_Block_offset_x = int(max(0, p_dist[0] - check_distance))
    min_Block_offset_y = int(max(0, p_dist[1] - check_distance))
    min_Block_offset_z = int(max(0, p_dist[2] - check_distance))
    max_Block_offset_x = int(min(p_dist[0] + 1 + check_distance, units_size[0]))
    max_Block_offset_y = int(min(p_dist[1] + 1 + check_distance, units_size[1]))
    max_Block_offset_z = int(min(p_dist[2] + 1 + check_distance, units_size[2]))
    
    for i in range(min_Block_offset_x, max_Block_offset_x):
        for j in range(min_Block_offset_y, max_Block_offset_y):
            for k in range(min_Block_offset_z, max_Block_offset_z):
                index = int(i + units_size[0] * j + units_size[0] * units_size[1] * k)
                for q in range(len(newgrid[index])):
                    distance = np.linalg.norm(p - points[newgrid[index][q]])
                    if distance < min_val:
                        closest_i = newgrid[index][q]
                        min_val = distance
    return closest_i

Define a function to set up the constraints for the input point set.

In [33]:
def set_constraints(p, ni, newgrid, units_size, grid_step):
    constrained_p = np.zeros((p.shape[0] * 3, 3))
    constrained_v = np.zeros((p.shape[0] * 3))

    size_p = p.shape[0]
    diag = igl.bounding_box_diagonal(p)
    eps = 0.01 * diag

    for i in range(size_p):
        constrained_p[i] = p[i]
        constrained_v[i] = 0
        constrained_p[i + size_p] = p[i] + eps * ni[i]
        while find_closed_point(p[i] + eps * ni[i], p, newgrid, units_size, grid_step) != i:
            eps *= 0.5
        constrained_p[i + size_p] = p[i] + eps * ni[i]
        constrained_v[i + size_p] = eps

        eps = 0.01 * diag
        constrained_p[i + 2 * size_p] = p[i] - eps * ni[i]
        while find_closed_point(p[i] - eps * ni[i], p, newgrid, units_size, grid_step) != i:
            eps *= 0.5
        constrained_p[i + 2 * size_p] = p[i] - eps * ni[i]
        constrained_v[i + 2 * size_p] = -eps 
    return constrained_p, constrained_v

Read the cat point cloud.

In [34]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)

Use the improved spatial search method to set up the constraints for the input point set.<br/> Display the constraints where the green, red and blue labels respectively correspond to inside, outside, and on the surface constructed by the point set.

In [35]:
step = 0.1
newgrid, units_size, grid_step = set_newgrid(step, pi)

T1 = time.time()
constrained_p, constrained_v = set_constraints(pi, ni, newgrid, units_size, grid_step)
T2 = time.time()
print('running time:%s ms' % ((T2 - T1)*1000))

r = np.array([1, 0, 0])
g = np.array([124/255, 252/255, 0])
b = np.array([0, 0, 1])
black = np.array([0, 0, 0])
color_map = np.zeros(constrained_p.shape)
for i in range(pi.shape[0]):
    color_map[i] = b
    color_map[i + pi.shape[0]] = r
    color_map[i + 2 * pi.shape[0]] = g

a = mp.plot(pi, c = np.zeros((pi.shape[0], 3)), shading={"point_size": 8})
mp.plot(constrained_p, c = color_map, shading = {"point_size": 8})

running time:268.1572437286377 ms


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

# Use MLS interpolation to extend to function _f_.

To improve the efficiency by implementing a simple spatial index, we bin vertices into their enclosing grid cells and restrict the neighbor queries to visit only the grid cells that could possibly satisfy the query.<br/>
Define a new method to accelerate neighbor calculations.

In [36]:
def closest_points(p, points, h, newgrid, units_size, grid_step):
    dist_index = []
    mmin = np.min(points, axis = 0)
    n_ori_points = int(points.shape[0] / 3)
    p_dist = np.zeros(3)
    p_dist[0] = math.floor(((p - mmin) / grid_step)[0])
    p_dist[1] = math.floor(((p - mmin) / grid_step)[1])
    p_dist[2] = math.floor(((p - mmin) / grid_step)[2])

    check_distance = 1
    min_Block_offset_x = int(max(0, p_dist[0] - check_distance))
    min_Block_offset_y = int(max(0, p_dist[1] - check_distance))
    min_Block_offset_z = int(max(0, p_dist[2] - check_distance))
    max_Block_offset_x = int(min(p_dist[0] + 1 + check_distance, units_size[0]))
    max_Block_offset_y = int(min(p_dist[1] + 1 + check_distance, units_size[1]))
    max_Block_offset_z = int(min(p_dist[2] + 1 + check_distance, units_size[2]))
    
    for i in range(min_Block_offset_x, max_Block_offset_x):
        for j in range(min_Block_offset_y, max_Block_offset_y):
            for k in range(min_Block_offset_z, max_Block_offset_z):
                index = int(i + units_size[0] * j + units_size[0] * units_size[1] * k)
                for q in range(len(newgrid[index])):
                    
                    distance = np.linalg.norm(p - points[newgrid[index][q]])
                    if distance < h:
                        dist_index.append(newgrid[index][q])
                        
                    distance = np.linalg.norm(p - points[n_ori_points + newgrid[index][q]])
                    if distance < h:
                        dist_index.append(n_ori_points + newgrid[index][q])
                        
                    distance = np.linalg.norm(p - points[n_ori_points * 2 + newgrid[index][q]])
                    if distance < h:
                        dist_index.append(n_ori_points * 2 + newgrid[index][q])
                    
    return dist_index

In [37]:
def wendland(r, h):
    if (r < h):
        return ((1 - r / h) ** 4) * (4 * r / h + 1)
    else:
        return 0

In [38]:
def polynomial(grid_point, degree):
    if degree == 0:
        basis = np.array([1])
    elif degree == 1:
        basis = np.array([1, grid_point[0], grid_point[1], grid_point[2]])
    elif degree == 2:
        basis = np.array([1, grid_point[0], grid_point[1], grid_point[2], grid_point[0] * grid_point[1], 
                          grid_point[1] * grid_point[2], grid_point[0] * grid_point[2], 
                          grid_point[0] ** 2, grid_point[1] ** 2, grid_point[2] ** 2])
    return basis

In [39]:
def closest_points_brute(p, points, h):
    dist_index = []
    for i in range(points.shape[0]):
        dist = np.linalg.norm(p - points[i])
        if dist < h:
            dist_index.append(i)
    return dist_index

In [40]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

MLS interpolation method to construct an implicit function satisfying the constraints as nearly as possible. 

In [41]:
def evaluate_implicit_f(grid_point, wendlandRadius, poly_degree, constrained_points, 
                        constrained_values, newgrid, units_size, grid_step):
    
    indices_in_h = closest_points(grid_point, constrained_points, wendlandRadius, newgrid, units_size, grid_step)
    indices_size = len(indices_in_h)
    n_coef = 3 ** poly_degree + 1
    if poly_degree == 0:
        n_coef = 1
    if(indices_size < n_coef * 2):
        return 10000
    else:
        B = np.zeros((indices_size, n_coef))
        W = np.eye(indices_size)
        constrained_point = constrained_points[indices_in_h]
        constrained_value = constrained_values[indices_in_h]
        for i in range(indices_size):
            B[i] = polynomial(constrained_point[i], poly_degree)
            W[i, i] = wendland(np.linalg.norm(grid_point - constrained_point[i], 2), wendlandRadius)

        ax = np.linalg.solve(((B.T).dot(W)).dot(B), ((B.T).dot(W)).dot(constrained_value))
        return polynomial(grid_point, poly_degree).dot(ax)

Create a regular volumetric grid around the point cloud: compute the axis-aligned bounding box of the point cloud, enlarge it slightly, and divide it into uniform cells (tets).

In [42]:
bbox_max = np.max(pi, axis = 0)
bbox_min = np.min(pi, axis = 0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min) 
res_x = 20
res_y = 20
res_z = 20
grid_points, tet = tet_grid((res_x, res_y, res_z), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

Evaluate the MLS function at every node of a regular volumetric grid containing the input point cloud.<br/>
The processing speed is way faster than using the point search method with brute-force for loop. We plot the grid points colored according to being inside or outside the input cloud.

In [43]:
polyDegree = 1
wendlandRadius = bbox_diag * 0.1
fx = np.array([evaluate_implicit_f(grid_point, wendlandRadius, polyDegree, constrained_p, constrained_v,
                                   newgrid, units_size, grid_step) for grid_point in grid_points])

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1

mp.plot(grid_points, c=ind, shading={"point_size": 4, "width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Using a non-axis-aligned grid

The point cloud luigi.off is not aligned with the canonical axes. Running reconstruction on an axis-aligned grid is wasteful in this case: many of the grid points will lie far outside the object.<br/> Here we devise an automatic (and general) way to align the grid to the data and implement it using PCA.

In [44]:
def PCA(points):
    mean = points - np.mean(points , axis = 0) 
    cov_mat = np.cov(mean, rowvar = False) 
    eigen_vals, eigenvectors = np.linalg.eig(cov_mat)     
    index_sorted = np.argsort(eigen_vals)[::-1]
    eigenvectors_sorted = eigenvectors[:,index_sorted]
    return eigenvectors_sorted.transpose()

Read the Luigi point cloud and make the transformation.

In [45]:
pi, v = igl.read_triangle_mesh("data/luigi.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)

In [60]:
b = mp.plot(pi, c = np.zeros((pi.shape[0], 3)), shading={"point_size": 0.5})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

In [46]:
eigenvectors_sorted = PCA(pi)
handle = np.array(((np.cos(np.pi/2), np.sin(np.pi/2), 0), (-np.sin(np.pi/2), np.cos(np.pi/2), 0), (0, 0, 1)))
new_pi = np.array([handle.dot(eigenvectors_sorted.dot(i)) for i in pi])
new_ni = np.array([handle.dot(eigenvectors_sorted.dot(i)) for i in ni])

To improve the efficiency, we still use a spaial index to accelerate neighbor calculations. Because Luigi object has way more points than cat, it takes longer time to process the point set.<br/> We slightly modify the new grid unit step to accelerate the processing.

In [47]:
step = 0.05
newgrid, units_size, grid_step = set_newgrid(step, new_pi)

T1 = time.time()
constrained_p, constrained_v = set_constraints(new_pi, new_ni, newgrid, units_size, grid_step)
T2 = time.time()
print('running time:%s ms' % ((T2 - T1)*1000))

running time:4326.792001724243 ms


In [48]:
bbox_max = np.max(new_pi, axis = 0)
bbox_min = np.min(new_pi, axis = 0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min) 
res_x = 30
res_y = 30
res_z = 30
grid_points, tet = tet_grid((res_x, res_y, res_z), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

We plot the grid with nodes colored according to their implicit function values.

In [49]:
T1 = time.time()

polyDegree = 1
wendlandRadius = bbox_diag * 0.1
fx = np.array([evaluate_implicit_f(grid_point, wendlandRadius, polyDegree, constrained_p, constrained_v,
                                   newgrid, units_size, grid_step) for grid_point in grid_points])
T2 = time.time()
print('running time:%s ms' % ((T2 - T1)*1000))

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1

mp.plot(grid_points, c=ind, shading={"point_size": 0.5, "width": 800, "height": 800})
#mp.plot(grid_points, c=fx, shading={"point_size": 0.5, "width": 800, "height": 800})

running time:64693.38893890381 ms


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.726429…

# Marching to extract surface

We use marching tets to extract the zero isosurface from the grid, filter out the artefacts and keep the largest component. Combining with the previous parts, we display multiple output with different parameters. It could be observed that the surface becomes finer as the resolution increases and Wendland function radius decreases.<br/> 
For your convenience to run and mark, I do not set all of the experiments with very large resolutions except the first two ones with the resolutions (40, 40, 40) of Luigi and (40, 45, 40) of cat. Given below experiments, it could be seen that my code outputs the appropriate results as expected, and every parameter works well.

Luigi: resolution = (40,40,40), polydegree = 1, Wendland function radius = 0.1 box diagonal.

In [50]:
sv, sf, _, _ = igl.marching_tets(grid_points, tet, fx, 0)

f = igl.face_components(sf)
components = []
for i in range(f.shape[0]):
    if f[i] not in components:
        components.append(f[i])
connected_components = np.zeros(len(components))
for i in range(f.shape[0]):
    connected_components[f[i]] += 1
new_sf = sf[f == np.argmax(connected_components)]
mp.plot(sv, new_sf, shading={"wireframe": True}) 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.739532…

In [51]:
def extract_surface(obj_address, step, res_x, res_y, res_z, polyDegree, wendland_radius_rate, filter = True):
    pi, v = igl.read_triangle_mesh(obj_address)
    pi /= 10
    ni = igl.per_vertex_normals(pi, v)   
    newgrid, units_size, grid_step = set_newgrid(step, pi)
    constrained_p, constrained_v = set_constraints(pi, ni, newgrid, units_size, grid_step)

    bbox_max = np.max(pi, axis = 0)
    bbox_min = np.min(pi, axis = 0)
    bbox_diag = np.linalg.norm(bbox_max - bbox_min) 
    grid_points, tet = tet_grid((res_x, res_y, res_z), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)
    
    wendlandRadius = wendland_radius_rate * bbox_diag    
    fx = np.array([evaluate_implicit_f(grid_point, wendlandRadius, polyDegree, constrained_p, constrained_v,
                                   newgrid, units_size, grid_step) for grid_point in grid_points])

    sv, sf, _, _ = igl.marching_tets(grid_points, tet, fx, 0)
    if filter:
        f = igl.face_components(sf)
        components = []
        for i in range(f.shape[0]):
            if f[i] not in components:
                components.append(f[i])
        connected_components = np.zeros(len(components))
        for i in range(f.shape[0]):
            connected_components[f[i]] += 1
        new_sf = sf[f == np.argmax(connected_components)]
        mp.plot(sv, new_sf, shading={"wireframe": True})       
    else:
        mp.plot(sv, sf, shading={"wireframe": True})

Cat: resolution = (40, 45, 40), polydegree = 1, Wendland function radius = 0.1 box diagonal.

In [52]:
extract_surface("data/cat.off", 0.06, 40, 45, 40, polyDegree = 1, wendland_radius_rate = 0.1, filter = True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.2277507…

Cat: resolution = (20,35,30), polydegree = 0, Wendland function radius = 0.1 box diagonal.

In [53]:
extract_surface("data/cat.off", 0.05, 20, 35, 30, polyDegree = 0, wendland_radius_rate = 0.1, filter = True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0034713…

Cat: resolution = (20,30,35), polydegree = 2, Wendland function radius = 0.2 box diagonal.

In [54]:
extract_surface("data/cat.off", 0.1, 20, 30, 35, polyDegree = 2, wendland_radius_rate = 0.2, filter = True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Cat: resolution = (25,30,25), polydegree = 1, Wendland function radius = 0.15 box diagonal.

In [55]:
extract_surface("data/cat.off", 0.1, 25, 30, 25, polyDegree = 1, wendland_radius_rate = 0.15, filter = True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0264654…